# Spojování, agregace a řazení

## Pokročilé úpravy



In [ ]:
import pandas as pd

url_food_nutrient = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/podmineny-vyber/podmineny-vyber/food_nutrient.csv"

food_nutrient = pd.read_csv(url_food_nutrient)
food_nutrient.head(3)

### Práce s chybějícími hodnotami



**`isna()`, `notna()`**

Obě metody vrací hodnoty `True` nebo `False`

In [ ]:
food_nutrient_incomplete = food_nutrient[food_nutrient["amount"].isna()]
food_nutrient_incomplete.head(3)

In [ ]:
food_nutrient_complete = food_nutrient[food_nutrient["amount"].notna()]
food_nutrient_complete.head(3)

**Jak chybějící hodnoty odstranit**

In [ ]:
food_nutrient = food_nutrient.dropna(subset="amount")
food_nutrient.head(3)

### Spojení dat

Nyní bychom chtěli všechny tři naše tabulky spojit do jedné. 

Nejprve si ukážeme, jak spojit tabulky pod sebe.

- Počet sloupců je ve výsledné tabulce stejný jako u spojovaných tabulek.
- Počet řádků odpovídá součtu řádků spojovaných tabulek.

V SQL používáme pro danou operaci klíčové slovo `UNION`, `pandas` používáme funkci `concat()`.

In [ ]:
url_food_sample_100 = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/nacteni-dat/nacteni-dat/food_sample_100.csv"
url_food_other = "https://kodim.cz/cms/assets/czechitas/python-data-1/python-pro-data-1/spojovani/pokrocile-upravy/food_other.csv"

food_sample_100 = pd.read_csv(url_food_sample_100)
food_other = pd.read_csv(url_food_other)

Pozor na to, že v takto vzniklém DataFrame se nám rozbije index, protože se spojí za sebe indexy jednotlivých tabulek. 

Pokud chceme, aby pandas při spojování index přepočítal, musíme nastavit hodnotu parametru `ignore_index` na `True`.

In [ ]:
food = pd.concat([food_sample_100, food_other], ignore_index=True)
food

# Propojení dat

`pandas` umí `DataFrame` také propojit, což odpovídá SQL příkazu `JOIN`. 

- Tabulky můžeme spojit více způsoby. 
- Výsledná tabulka bude mít více sloupců.
- Počet řádků zavisí na typu propojení.


![Druhy propojeni](../../img/type_of_join_operations.png)

U operace `JOIN` jsou důležité dvě věci:

- Podle jakého sloupce (nebo jakých sloupců) dvě různé tabulky propojujeme.
- Co udělat v případě, že pro nějaké řádky nemám ve druhé tabulce odpovídající hodnotu.

In [ ]:
print(food.shape)
print(food_nutrient.shape)

In [ ]:
food_merged = pd.merge(food, food_nutrient, on="fdc_id", how="inner")
food_merged.shape

---
# Cvičení


## Značky a výrobci

Stáhni si data z tabulky `branded_food.csv`, která obsahuje data o konkrétních potravinách od výrobců. Tabulku načti do `pandas` tabulky `branded_food`.

Pro tabulku `branded_food` splň následující úkoly.

_Nápověda: Na body 2 a 3 využij metodu, kterou jsme si již ukazovali v předchozí lekci. Její název se skládá ze dvou slov - anglického výrazu pro hodnotu a pro počet._



1. Zobraz si prvních několik řádků tabulky a podívej se na to, jaké jsou v ní sloupce a jaké jsou v nich hodnoty.



2. Ve sloupci `brand_owner` jsou názvy výrobců potravin. Zjisti tři výrobce s největším počtem potravin v tabulce.




3. Ve sloupci `branded_food_category` jsou kategorie potravin. Zjisti pět kategorií s největších počtem potravin v tabulce.


V tabulce je sloupec `fdc_id`, pomocí kterého ji můžeš propojit s tabulkou `food_merged`. 

Protože názvy jsou v obou tabulkách stejné, takže by bylo možné použít parametr `on`. Vyzkoušej si ale místo toho parametry `left_on` a `right_on`, kterým dáš stejnou hodnotu, tj. název sloupce `fdc_id`. 

Výsledek ulož do tabulky `food_merged_brands`.



Pro tabulku `food_merged_brands` splň následující úkoly.

1. U výsledné tabulky `food_merged_brands` zkontroluj počet řádků a srovnej ho s původní tabulkou `food_merged`. Ubyly nějaké řádky? A čím to je?


2. Nyní proveď operaci merge znovu, ale s parametrem `how` nastaveným na hodnotu `left`. Zkontroluj počet řádků a porovnej ho s počtem řádků tabulky `food_merged_brands`. Proč se počet liší?

## Přejmenování sloupce

Čím více tabulky propojujeme, tím více sloupců v nich máme. To může být časem až matoucí. 

Například máme v tabulce sloupec name, který označuje název výživné látky. 

Pro větší přehlednost ho přejmenujme na `nutrient_name`. K tomu slouží metoda `rename`. 

Níže je příklad toho, jak lze metodu použít. Pozorně si ho prohlédni. 

Metodu voláme pomocí tečkové notace, kterou už jsme využívali. 

```food_merged_brands = food_merged_brands.rename(columns={"stary_nazev": "novy_nazev"})```


Dále je potřeba zapsat název parametru (`columns`, protože chceme přejmenovat sloupce). Jako hodnotu vkládáme strukturu, která je označováno jako slovník. Jedná se o dvojici hodnot ve složených závorkách, která je oddělená dvojtečkou. Před dvojtečkou je starý název sloupce (v našem případě to bude name) a za dvojtečkou nový název sloupce (v našem případě to bude `nutrition_name`). Uprav tedy použití metody a spusť ho. Následně zkontroluj výsledek pomocí `.columns`.



Jako bonus se zamysli nad tím, co by se mohlo stát, pokud na místo starého názvu zadáš sloupec, který v tabulce není (např. test). 

Jaké chování bys od metody očekával(a)? Jakmile to budeš mít rozmyšlené, použij metodu s neexistujícím názvem sloupce a zkontroluj, co se stalo.

# Odstranění sloupců

Pokud máme v tabulce zbytečné sloupce, můžeme je odstranit. 

Zbytečných sloupců se můžeme zbavit dvě způsoby:

1. Výběrem sloupců, které v tabulce chceme. To už jsme si ukazovali v předchozí lekci v části "Výběr sloupců".
2. Odebráním sloupců, které v tabulce nechceme. K tomu slouží metoda `


Níže je příklad použití metody `drop()`. Použijeme parametr `columns`, kterému zadáme seznam sloupců k odstranění.


```
food_merged_brands = food_merged_brands.drop(columns=["sloupec_1", "sloupec_2"])
```


Využij metodu k odstranění sloupců `footnote` a `min_year_acquired`. 



Pomocí vlastnosti `.columns` zkontroluj výsledek. 



Nakonec se podobně jako v předchozím cvičení zamysli nad tím, co by se mohlo stát, kdybys do seznamu vložila neexistující název sloupce, například test. 

Poté metodu `drop()` s neexistujícím názvem sloupce vyzkoušej. Překvapil tě výsledek?

## Agregace a řazení



### Agregace
Agregace je operace, která "sloučí" více řádků tabulky do jednoho. 

U agregace většinou slučujeme řádky podle nějakého konkrétního sloupce. 

In [ ]:
food_merged_brands_protein = food_merged_brands[
    food_merged_brands["nutrient_name"] == "Protein"
]
food_merged_brands_protein.head(3)

In [ ]:
food_merged_brands_protein_agg = food_merged_brands_protein.groupby(
    "branded_food_category"
)["amount"].mean()

food_merged_brands_protein_agg

### Řazení

In [ ]:
food_merged_brands_protein_agg.sort_values(ascending=False).head(10)

In [ ]:
food_merged_brands_protein.sort_values("amount", ascending=False).head()

---

# Cvičení

## Lipidy a tuky

Podívej se nyní na to, které kategorie potravin obsahují nejvíce lipidů (tuků). 

Nejprve pomocí dotazu vytvořit novou tabulku `food_merged_brands_lipid`, do které pomocí dotazu vlož pouze řádky, které mají jako název výživné látky hodnotu `Total lipid (fat)`. 


Poté proveď agregaci podle návu kategorie a seřaď výslednou tabulku tak, aby nahoře byly vidět kategorie s největším počtem tuků. 

Porovnej si výslednou tabulku s tabulkou `food_merged_brands_protein_agg`, kterou jsme vytvořili v rámci lekce. 

Podívej se, zda se některé kategorie objevují v obou tabulkách.

## Výrobci a kategorie

Nyní uvažuj, že si chceme udělat přehled o tom, jaký výrobce produkuje jaké typy potravin. 



Proveď agregaci tabulky `food_merged_brands` podle dvou sloupců: `brand_owner` a `branded_food_category`. 

Sloupce musíš metodě `food_merged_brands` zadat jako seznam, tj. musíš použít hranaté závorky. 




Dále vyber sloupec `fdc_id` pro provedení agregace a použij agregaci `nunique()`, který vrátí počet unikátních hodnot. 

Nakonec použij metodu `sort_values` s tím, že chceš data seřadit sestupně.